# Накопленный охват в динамике по ФО
Пример расчета показателей телеканалов в разбивке по федеральным округам и месяцам.

## Описание задачи и условий расчета
- Период: 01.01.2022 - 30.09.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 4+ в разбивке по федеральным округам
- Место просмотра: Все места (дом+дача)
- Каналы: ПЕРВЫЙ КАНАЛ									
- Статистики: Reach%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

In [ ]:
# выберем телесети по имени (части имени)
cats.get_tv_net(name=["ПЕРВЫЙ КАНАЛ"])

# далее в запросах будем использовать id телесетей.

Обратившись к каталогу демографических переменных, получим entityName переменной Федеральные округа

In [ ]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['федерал'])

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-01-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId = 1'

# Указываем список статистик для расчета
statistics = ['ReachPer']

# Указываем срезы
slices = ['federalOkrug', 'researchMonth']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

## Расчет задания

In [ ]:
# Формируем задание для TV API в формате JSON
task_json = mtask.build_task('timeband', date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))
df

Настраиваем внешний вид таблицы

In [ ]:
df_res = pd.pivot_table(df, values=['ReachPer'],
                        index=['federalOkrug'], 
                        columns=['researchMonth'])
df_res

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('02_reach_by_federal_okrug')) as writer:
    df_res.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)